# DNN

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

## 데이터 가져오기

In [2]:
train_set = pd.read_csv('train_set.csv')
train_set = train_set[train_set['ECLO'] <= 25]
test_set = pd.read_csv('test_set.csv')

## train test valid 분리

In [3]:
train_x, test_x, train_y, test_y = train_test_split(train_set[['요일', '기상상태', '도로형태', '노면상태', '사고유형']], train_set['ECLO'], test_size=0.2)

In [4]:
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2)

## tensorflow model 정의

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(5)),
    tf.keras.layers.Dense(5, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(10, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(50, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(100, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(50, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(10, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(1)]
)

In [10]:
checkpoint_filepath = "logs/DNN/"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
)

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredLogarithmicError(), metrics=['mae'])
model.fit(train_x, train_y, epochs=100, validation_data=(valid_x, valid_y),
          callbacks=[
                        checkpoint_callback,
                        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3),
                    ])

Epoch 1/100
791/791 [==============================] - 11s 11ms/step - loss: 0.3623 - mae: 2.5504 - val_loss: 0.2145 - val_mae: 2.0278
Epoch 2/100
791/791 [==============================] - 8s 10ms/step - loss: 0.2343 - mae: 2.1271 - val_loss: 0.2164 - val_mae: 2.0261
Epoch 3/100
791/791 [==============================] - 8s 11ms/step - loss: 0.2114 - mae: 2.0478 - val_loss: 0.2071 - val_mae: 2.0404
Epoch 4/100
791/791 [==============================] - 8s 11ms/step - loss: 0.2031 - mae: 2.0386 - val_loss: 0.2111 - val_mae: 2.0354
Epoch 5/100
791/791 [==============================] - 8s 11ms/step - loss: 0.2002 - mae: 2.0380 - val_loss: 0.2045 - val_mae: 2.0496
Epoch 6/100
791/791 [==============================] - 8s 10ms/step - loss: 0.1995 - mae: 2.0383 - val_loss: 0.2043 - val_mae: 2.0498
Epoch 7/100
791/791 [==============================] - 8s 11ms/step - loss: 0.1993 - mae: 2.0386 - val_loss: 0.2020 - val_mae: 2.0523
Epoch 8/100
791/791 [==============================] - 9s 11m

In [12]:
np.sqrt(mean_squared_log_error(test_y, model.predict(test_x)))

248/248 [==============================] - 1s 3ms/step


0.4435358029685519

## 결과 저장

In [13]:
ECLOs = model.predict(test_set[['요일', '기상상태', '도로형태', '노면상태', '사고유형']])
ans = pd.read_csv('open/sample_submission.csv')
ans['ECLO'] = ECLOs
ans.to_csv('ans_dnn.csv', index=False)

343/343 [==============================] - 1s 3ms/step
